$$ ITI \space AI-Pro: \space Intake \space 43 $$
$$ Recommender \space Systems $$
$$ Lab \space no. \space 3 $$

# `01` Import Necessary Libraries

## `i` Default Libraries

In [1]:
import numpy as np
import pandas as pd

## `ii` Additional Libraries
Add imports for additional libraries you used throughout the notebook

In [18]:
from surprise.reader import Reader
from surprise.dataset import Dataset
from surprise.model_selection import train_test_split
from surprise.prediction_algorithms.matrix_factorization import SVD
from surprise.accuracy import rmse,mse,mae

----------------------------

# `02` Load Data

Load `songsModifiedDataset.csv` file into a dataframe

In [2]:
data = pd.read_csv("Data/songsModifiedDataset.csv")
data.head()

,userID,songID,rating
0,0,90409,5
1,4,91266,1
2,5,8063,2
3,5,24427,4
4,5,105433,4


--------------------------

# `03` Matrix Factorization using Gradient Descent

Practice for Matrix Factorization Implementation

**Matrix Factorization Mathematical Derivation**

We know that the matrix factorization breaks the rating matrix $R$ into two matrices $U \in \mathbb{R}^{\#users \times K}$ and $M \in \mathbb{R}^{K \times \#items}$ where K represent the latent space dimensionality.

$R$ can then be approximated through the following equation:

$$
\mathbf{R} \approx \mathbf{U} \times \mathbf{M} = \hat{\mathbf{R}}
$$

The error, incorporating the regularization term, is calculated as follows:

$$
e_{ij}^2 = (r_{ij} - \sum_{k=1}^K{u_{ik}m_{kj}})^2 + \frac{\beta}{2} \sum_{k=1}^K{(||U||^2 + ||M||^2)}
$$

In order to be able to use Stochastic Gradient Descent (SGD) to optimize $U$ and $M$, we need to find the partial derivatives of the error function with respect to both $u_{ik}$ and $m_{kj}$. The partial derivatives can be derived as follows:

$$
\frac{\partial}{\partial u_{ik}}e_{ij}^2 = -2(r_{ij} - \hat{r}_{ij})(m_{kj}) + \frac{\beta}{2} \times (2 u_{ik}) = -2 e_{ij} m_{kj} + \beta u_{ik} 
$$

$$
\frac{\partial}{\partial m_{ik}}e_{ij}^2 = -2(r_{ij} - \hat{r}_{ij})(u_{ik}) + \frac{\beta}{2} \times (2 m_{kj}) = -2 e_{ij} u_{ik} + \beta m_{kj}
$$

Thus the update rules will be:

$$
u'_{ik} = u_{ik} + \alpha \frac{\partial}{\partial u_{ik}}e_{ij}^2 = u_{ik} - \alpha(-2 e_{ij} m_{kj} + \beta u_{ik} ) = u_{ik} + \alpha(2 e_{ij} m_{kj} - \beta u_{ik} )
$$

$$
m'_{kj} = m_{kj} + \alpha \frac{\partial}{\partial m_{kj}}e_{ij}^2 = m_{kj} - \alpha(-2 e_{ij} u_{ik} + \beta m_{kj} ) = m_{kj} + \alpha(2 e_{ij} u_{ik} - \beta m_{kj} )
$$

## `0` Construct Utility Matrix from the Data

In [3]:
utility_matrix = data.pivot(index='userID', columns='songID',values='rating')
utility_matrix = utility_matrix.fillna(0)
utility_matrix

songID,2263,2726,3785,8063,12709,13859,16548,17029,19299,19670,...,113954,119103,120147,122065,123176,125557,126757,131048,132189,134732
userID,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199976,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0
199980,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199988,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## `i` Matrices Initialization

Initialize the two random weights matrices $U$ and $M$ (Try $K=3$)

**Note**: Refer to the next cell for the dimensions of $U$ and $M$

**Hine**: You may use a function from `numpy.random` module (see, [Documentation](https://numpy.org/doc/stable/reference/random/index.html))

In [69]:
k=3
no_of_users = utility_matrix.shape[0]
no_of_items = utility_matrix.shape[1]

U_random = np.random.random((no_of_users, k))
M_random = np.random.random((k, no_of_items))

## `ii` Define a Function to Implement Matrix Factorization

**Function Parameters**:
- `R`: Utility Matrix [of shape: (no. of users, no. of items)]
- `U`: User Latent Features Array [of shape: (no. of users, K)]
- `M`: Items Latent Features Array [of shape: (K, no. of items)]
- `epochs`: No. of training epochs
- `lr`: Learning rate (alpha)
- `beta`: Regularization Parameter

**Function Output**:
- `U`: Optimized User Latent Features Array
- `M`: Optimized Items Latent Features Array

**Main Procedures**:
1. Calculate predicted ratings
2. Calculate MSE Error
3. Calculate gradients
4. Update $U$ and $M$


In [79]:
def matrix_factorization(R: np.ndarray, U: np.ndarray, M: np.ndarray, epochs: int, lr: float, beta: float):
    """
    Function Parameters:
    - `R`: Utility Matrix [of shape: `(no. of users, no. of items)`]
    - `U`: User Latent Features Array [of shape: `(no. of users, K)`]
    - `M`: Items Latent Features Array [of shape: `(K, no. of items)`]
    - `epochs`: No. of training epochs
    - `lr`: Learning rate (alpha)
    - `beta`: Regularization Parameter

    Function Output:
    - `U`: Optimized User Latent Features Array
    - `M`: Optimized Items Latent Features Array
    """
    U = U.copy()
    M = M.copy()
    
    # Confirm that no. of features is consistent between U and M
    assert U.shape[1] == M.shape[0], f'U and M must have consistent K. Found K={U.shape[1]} for U and K={M.shape[0]} for M'

    # Extract No. of Features (K)
    K = M.shape[0]

    # Define the Epochs loop
    for epoch in range(epochs):
        # Loop over every element in R
        for i in range(U.shape[0]): # Loop over each user
            for j in range(M.shape[1]): # Loop over each item
                if R[i,j] !=0: # Only proceed if the current interaction (i, j) is not missing
                    eij =  R[i,j] - (U[i] @ M.T[j])  # Calculate the error in prediction
                    for d in range(K): # Loop over each latent features dimension
                        # Update Rules for both U and M:
                        U[i, d] = U[i, d] + lr*(2*eij*M[d, j] - beta * U[i, d])
                        M[d, j] = M[d, j] + lr*(2*eij*U[i, d] - beta * M[d, j])

        ## Error Calculation ##
        e_last = e if epoch > 0 else 100000000
        e = 0 # Initialize a variable to accumelate the errors
        for i in range(U.shape[0]): # Loop over each user
            for j in range(M.shape[1]): # Loop over each item
                if R[i,j] !=0: # Only proceed if the current interaction (i, j) is not missing
                                # since we only calculate the error for interactions having a ground truth value

                    first_part =(U[i] @ M.T[j] - R[i,j]) ** 2  # calculate the first part of the error
                    second_part = 0 # Initialize a variable to accumelate the second part of the error

                    for d in range(K): # Loop over each latent features dimension
                        second_part += ((beta / 2) * (U[i,d] ** 2 + M[d,j] ** 2))

                    e += first_part + second_part # accumelate the error to the total error
                
        print(f'Epoch {epoch+1}/{epochs}: Total Error = {e}')

        if e < 0.001 or e_last-e < 0.001: # Stop if error is so small or improvement is not significant
            break

    return U, M


### `[Bonus]` Vectorized Error Calculation

Can the error calculation part be vectorized to get rid of for loops?

If you would like a challenge, try to redefine the function in the next cell with a vectorized error calculation.

In [84]:
def matrix_factorization_vectorized(R: np.ndarray, U: np.ndarray, M: np.ndarray, epochs: int, lr: float, beta: float):
    U = U.copy()
    M = M.copy()

    assert U.shape[1] == M.shape[0], f'U and M must have consistent K. Found K={U.shape[1]} for U and K={M.shape[0]} for M'

    nonzero_indices = np.nonzero(R)

    for epoch in range(epochs):
        # Calculate error matrix E
        E = R - np.dot(U, M)
        E[R == 0] = 0

        # Update U and M
        U_grad = 2 * np.dot(E, M.T) - beta * U
        M_grad = 2 * np.dot(U.T, E) - beta * M

        np.clip(U_grad, -1e10, 1e10, out=U_grad)
        np.clip(M_grad, -1e10, 1e10, out=M_grad)

        U += lr * U_grad
        M += lr * M_grad
        
        # Calculate total error
        first_part = np.sum(E ** 2)
        second_part = beta * (np.sum(U ** 2) + np.sum(M ** 2))
        error = first_part + second_part

        print(f'Epoch {epoch+1}/{epochs}: Total Error = {error}')

        # Check for convergence
        if error < 0.001:
            break

    return U, M

## `iii` Use the Function to to Optimize the $U$ and $V$

In [72]:
U_1, M_1 = matrix_factorization(R=utility_matrix.values, U=U_random, M=M_random, epochs=50, lr=0.01, beta=0.001)

Epoch 1/50: Total Error = 118328.96623864437
Epoch 2/50: Total Error = 65556.81906881601
Epoch 3/50: Total Error = 43876.47974432108
Epoch 4/50: Total Error = 34022.82070152592
Epoch 5/50: Total Error = 28825.536703142392
Epoch 6/50: Total Error = 25448.708844733414
Epoch 7/50: Total Error = 22791.981503199026
Epoch 8/50: Total Error = 20475.56106691269
Epoch 9/50: Total Error = 18401.72922077483
Epoch 10/50: Total Error = 16561.48910933765
Epoch 11/50: Total Error = 14958.100364405602
Epoch 12/50: Total Error = 13584.168417720413
Epoch 13/50: Total Error = 12419.629809713353
Epoch 14/50: Total Error = 11436.367367445302
Epoch 15/50: Total Error = 10603.823076813811
Epoch 16/50: Total Error = 9893.413042569317
Epoch 17/50: Total Error = 9281.143694380215
Epoch 18/50: Total Error = 8748.575276485159
Epoch 19/50: Total Error = 8282.531344680638
Epoch 20/50: Total Error = 7874.012267488652
Epoch 21/50: Total Error = 7516.769987634867
Epoch 22/50: Total Error = 7205.93978904124
Epoch 23/50

In [85]:
U_2, M_2 = matrix_factorization_vectorized(R=utility_matrix.values, U=U_random, M=M_random, epochs=50, lr=0.01, beta=0.001)

Epoch 1/50: Total Error = 694104.5453012063
Epoch 2/50: Total Error = 243354930.5059271
Epoch 3/50: Total Error = 3.4402738806286736e+16
Epoch 4/50: Total Error = 2.2655935445395293e+36
Epoch 5/50: Total Error = 4.347199398243441e+34
Epoch 6/50: Total Error = 2.2655935445395293e+36
Epoch 7/50: Total Error = 4.347199398243441e+34
Epoch 8/50: Total Error = 2.2655935445395293e+36
Epoch 9/50: Total Error = 4.347199398243441e+34
Epoch 10/50: Total Error = 2.2655935445395293e+36
Epoch 11/50: Total Error = 4.347199398243441e+34
Epoch 12/50: Total Error = 2.2655935445395293e+36
Epoch 13/50: Total Error = 4.347199398243441e+34
Epoch 14/50: Total Error = 2.2655935445395293e+36
Epoch 15/50: Total Error = 4.347199398243441e+34
Epoch 16/50: Total Error = 2.2655935445395293e+36
Epoch 17/50: Total Error = 4.347199398243441e+34
Epoch 18/50: Total Error = 2.2655935445395293e+36
Epoch 19/50: Total Error = 4.347199398243441e+34
Epoch 20/50: Total Error = 2.2655935445395293e+36
Epoch 21/50: Total Error = 

## `iv` Recommend top-K Songs

Recommend top-K ($K=5$) songs for user ($userID=199988$)

Note: Make sure to filter songs they already rated before

In [112]:
# Find the index of user 199988 in the U matrix
user_id = 199988
user_ids = utility_matrix.index
user_idx = np.where(user_ids == user_id)[0][0]

# Compute predicted ratings for user 199988
predicted_ratings = np.dot(U_1[user_idx], M_1)

# Find the indices of already rated items
rated_items = np.nonzero(utility_matrix.loc[user_id].values)[0]

# Set predicted ratings of already rated items to -1 (to exclude them from top-K recommendations)
predicted_ratings[rated_items] = -1

# Get the indices and values of top-K items
top_K_item_ids = np.argsort(predicted_ratings)[-5:][::-1]
top_K_item_ratings = np.sort(predicted_ratings)[-5:][::-1]

print(f"Top-5 recommendations for User {user_id}:")
for i,(id,rating) in enumerate(zip(utility_matrix.columns[top_K_item_ids],top_K_item_ratings)):
    print(f"\t -Top {i+1} Song: {id} (Predicted Rating: {rating})")

Top-5 recommendations for User 199988:
	 -Top 1 Song: 126757 (Predicted Rating: 6.240960809348716)
	 -Top 2 Song: 55622 (Predicted Rating: 6.19832767926378)
	 -Top 3 Song: 13859 (Predicted Rating: 6.092746850453148)
	 -Top 4 Song: 62954 (Predicted Rating: 5.959047906678433)
	 -Top 5 Song: 42781 (Predicted Rating: 5.9105081180136265)


------------------------

# `04` Matrix Factorization using SVD Algorithm

Practice for using `SVD` algorithm implementation from `scikit surprise` library.

## `i` Prepare the Data

- Load the Data into `surprise` Dataset
- Split data into train and test


In [11]:
reader = Reader()
dataset = Dataset.load_from_df(data, reader)

In [13]:
train_data, test_data = train_test_split(dataset, test_size=0.2)

## `ii` Model Initialization

Instantiate two models:
- Model with baselines (biases)
- Model without baselines

**Note**: Use `surprise.prediction_algorithms.matrix_factorization.SVD` (see, [Documentation](https://surprise.readthedocs.io/en/stable/matrix_factorization.html#:~:text=surprise.prediction_algorithms.matrix_factorization.-,SVD,-(n_factors%3D)))

In [14]:
# Biased Model
# 100 factors and 20 iterations as default
model_1= SVD()

In [15]:
# Non-Biased Model
model_2 = SVD(biased=False)

## `iii` Fit each Model on Training Data

In [16]:
# Biased Model
model_1.fit(train_data)

In [17]:
# Non-Biased Model
model_2.fit(train_data)

## `iv` Test both Models on the Testing Data

Compare the errors of the two models using multiple error formulas.

**Note**: Refer to `surprise.accuracy` module (see, [Documentation](https://surprise.readthedocs.io/en/stable/accuracy.html))

In [22]:
# Biased Model
predictions_1 = model_1.test(test_data)
print(rmse(predictions_1))
print(mse(predictions_1))
print(mae(predictions_1))

RMSE: 1.4977
1.4976651515720272
MSE: 2.2430
2.243000906233263
MAE:  1.2909
1.2909303690353833


In [23]:
# Non-Biased Model
predictions_2 = model_2.test(test_data)
print(rmse(predictions_2))
print(mse(predictions_2))
print(mae(predictions_2))

RMSE: 2.0653
2.0652833067518332
MSE: 4.2654
4.265395137147787
MAE:  1.7184
1.718421191278021


## `v` Recommend Top $10$ Songs for User $199988$

Is there a difference in recommended songs from the two models?

In [49]:
users_1 = np.array([item.uid for item in predictions_1]).reshape(-1,1)
items_1 = np.array([item.iid for item in predictions_1]).reshape(-1,1)
real_1 = np.array([item.r_ui for item in predictions_1]).reshape(-1,1)
predicted_1 = np.array([item.est for item in predictions_1]).reshape(-1,1)

modified_predicted_1 = np.concatenate([users_1,items_1,real_1,predicted_1],axis=1)


In [117]:
# Biased Model
User_ID = 199988
df_biased = pd.DataFrame(modified_predicted_1,columns=['UserID','SongID','real_rating','Predicted_rating'])
df_biased.drop(columns=['UserID'],inplace=True)

#df_filtered = df_biased.loc[(df_biased['UserID'] == User_ID)]
#df_filtered_sorted = df_filtered.sort_values('Predicted_rating', ascending=False)
#df_filtered_sorted

df_biased_sorted = df_biased.sort_values('Predicted_rating', ascending=False)
df_biased_sorted.head(10)

,SongID,real_rating,Predicted_rating
6565,71582.0,4.0,5.000000
10166,52611.0,5.0,5.000000
12824,13859.0,5.0,5.000000
11089,71582.0,5.0,4.959347
4260,13859.0,4.0,4.938730
2777,55240.0,5.0,4.927425
11080,122065.0,5.0,4.924872
6576,42781.0,5.0,4.922049
4100,55240.0,3.0,4.891425
8564,52611.0,4.0,4.876822


In [57]:
users_2 = np.array([item.uid for item in predictions_2]).reshape(-1,1)
items_2 = np.array([item.iid for item in predictions_2]).reshape(-1,1)
real_2 = np.array([item.r_ui for item in predictions_2]).reshape(-1,1)
predicted_2 = np.array([item.est for item in predictions_2]).reshape(-1,1)

modified_predicted_2 = np.concatenate([users_2,items_2,real_2,predicted_2],axis=1)

In [118]:
# Non-Biased Model
User_ID = 199988
df_non_biased = pd.DataFrame(modified_predicted_2,columns=['UserID','SongID','real_rating','Predicted_rating'])
df_non_biased.drop(columns=['UserID'],inplace=True)

#df_filtered = df_non_biased.loc[(df_non_biased['UserID'] == User_ID)]
#df_filtered_sorted = df_filtered.sort_values('Predicted_rating', ascending=False)
#df_filtered_sorted

df_non_biased_sorted = df_non_biased.sort_values('Predicted_rating', ascending=False)
df_non_biased_sorted.head(10)

,SongID,real_rating,Predicted_rating
6827,122065.0,5.0,4.697744
11151,125557.0,5.0,3.473426
0,28985.0,2.0,3.453293
8551,19299.0,4.0,3.453293
8541,19670.0,1.0,3.453293
8542,123176.0,4.0,3.453293
8543,19299.0,4.0,3.453293
8545,123176.0,5.0,3.453293
8546,16548.0,1.0,3.453293
8549,94535.0,5.0,3.453293


----------------------------------------------

$$ Wish \space you \space all \space the \space best \space ♡ $$
$$ Abdelrahman \space Eid $$